In [ ]:
import sys 
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
import dateutil.parser as dp
import datetime
import pyasn
import re
import os
import ipaddress

Global variables

In [2]:
_ASN_FILES = './db.rviews/'
ASN_DB_CACHE = {}

Method for finding closes matching database file

In [3]:

def closest_matching_asndb(date):
    """ Returns Closest Matching ASNDB for a specific day looking forward in time"""
    #construct the path you want to check for
    s = os.path.join(_ASN_FILES, 'ipasn_%4d%02d%02d.dat' % (date.year, date.month, date.day))
    if os.path.isfile(s):
        return s
    else:
        # check if the desired path is either forwards in time or backwards in time and return if found

        # look forwards in time to match
        d = date + datetime.timedelta(days=1)
        while d <= datetime.date.today():
            s = os.path.join(_ASN_FILES, 'ipasn_%4d%02d%02d.dat' % (d.year, d.month, d.day))
            if os.path.isfile(s):
                return s
            d += datetime.timedelta(days=1)
        
        # Didn't find any up to date file that is newer
        # Lets work with what we have and use the latest old file
        # by looking backwards in time to match

        d = date - datetime.timedelta(days=1)
        while d >= dp.parse("2013-01-01").date():
            s = os.path.join(_ASN_FILES, 'ipasn_%4d%02d%02d.dat' % (d.year, d.month, d.day))
            if os.path.isfile(s):
                return s
            d -= datetime.timedelta(days=1)

    return None

Method for finding an organisation

In [4]:
def lookup_asn_prefix(asndb, ip):
    ip_str = str(ip)
    asn = asndb.lookup(ip_str)
    if asn is None:
        asn = "None"
    return asn

Method for looking up an ip

In [9]:
def lookup_ip(date, ip):
    #print(('lookup_ip {0} and {1}').format(date, ip))
    asn = None
    
    #date you want to find, parsed to a date format
    probe_date = dp.parse(date).date()

    #Check if the probe_date is not already in the cache
    if probe_date not in ASN_DB_CACHE:
        #If it is not in the cache
        matching_asn = closest_matching_asndb(probe_date)
        ASN_DB_CACHE[probe_date] = pyasn.pyasn(matching_asn)
    try:
        asn = lookup_asn(ASN_DB_CACHE[probe_date], ip)
    except:
        pass
    return asn

In [10]:
def lookup_as_ip_ranges(date, lower, upper):
    ipIndex = ipaddress.ip_address(lower)
    ipUpperBound = ipaddress.ip_address(upper)
    resevedCounter = 0
    orgDict = {}
    
    while ipIndex < ipUpperBound:
        #if the ip is in a reserved network, change it to last reserved element  + 1
        reservedLower = resNetObj[resevedCounter]
        if ipIndex == reservedLower[0]:
            ipIndex = reservedLower[-1] + 1
            resevedCounter += 1
        else:
            org = lookup_ip(date, ipIndex)
            if org is not 'None':
                orgName = org[0]
                orgRange = org[1]
                if orgName in orgDict:
                    orgDict[orgName].append(org[1])
                else:
                    orgDict[orgName] = [org[1]]
                ipIndex = ipaddress.ip_address(orgRange[1]) + 1  
            else:
                ipIndex += 1
    print('organizations found on selected date:')
    for k in orgDict:
        print(k)
#         print(('Organization: {0}').format(k))
#         for v in orgDict[k]:
#             print(v)

Running functions

In [8]:
lookup_as_ip_ranges("2018-03-05", '8.8.8.8')

(15169, '8.8.8.0/24')